<h1>Data Preprocessing Notebook for Covid Diet Project</h1>

In [1]:
#Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler

#Performing Principal Component Analysis on Fat Supply, Protien Supply, kcal intake, and kg supply.
#Given that our data is so high dimensional, we wish to see what features are the most imoortant among all features.

protein_dataset = pd.read_csv('data/Protein_Supply_Quantity_Data.csv')
kcal_dataset = pd.read_csv('data/Food_Supply_kcal_Data.csv')
kg_dataset = pd.read_csv('data/Food_Supply_Quantity_kg_Data.csv')
fat_dataset = pd.read_csv('data/Fat_Supply_Quantity_Data.csv')
#Note, each column before obesity & undernourished sums up to 100%

In [2]:
#prepare the dataframes for merging

#drop the covid columns that are the same in every dataframe, except for one dataframe
#remove Unit (all except Population) column, as it is totally useless
kcal_dataset=kcal_dataset.drop(['Obesity', 'Undernourished','Confirmed', 'Deaths', 'Recovered', 'Active', 'Population', 'Unit (all except Population)'],axis=1)
kg_dataset=kg_dataset.drop(['Obesity', 'Undernourished','Confirmed', 'Deaths', 'Recovered', 'Active', 'Population', 'Unit (all except Population)'],axis=1)
fat_dataset=fat_dataset.drop(['Obesity', 'Undernourished', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Population', 'Unit (all except Population)'],axis=1)
protein_dataset=protein_dataset.drop(['Unit (all except Population)'],axis=1)

In [3]:
#append an identifier at the end of each dataframe row to show where it came from
for (columnName, columnData) in kcal_dataset.iteritems(): 
    if(columnName!="Country"):
        kcal_dataset.rename(columns={columnName: columnName+"_kcal"}, inplace=True)
for (columnName, columnData) in fat_dataset.iteritems():
    if(columnName!="Country"):
        fat_dataset.rename(columns={columnName: columnName+"_f"}, inplace=True)
for (columnName, columnData) in kg_dataset.iteritems(): 
    if(columnName!="Country"):
        kg_dataset.rename(columns={columnName: columnName+"_kg"}, inplace=True)
keepColumns = ['Country','Obesity', 'Undernourished', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Population']
for (columnName, columnData) in protein_dataset.iteritems(): 
    rename=True
    for keepName in keepColumns:
        if(columnName==keepName):
            rename=False
    if(rename):
        protein_dataset.rename(columns={columnName: columnName+"_p"}, inplace=True)

In [4]:
#finally, merge the dataframes
merged_df = kcal_dataset.merge(fat_dataset, left_on='Country', right_on='Country')
merged_df = merged_df.merge(protein_dataset, left_on='Country', right_on='Country')
merged_df = merged_df.merge(kg_dataset, left_on='Country', right_on='Country')

In [5]:
#taking a look at the undernourished column, we can see that there is a strange value: <2.5
merged_df['Undernourished'].unique()

array(['29.8', '6.2', '3.9', '25', nan, '4.6', '4.3', '<2.5', '14.7',
       '7.5', '10.1', '17.1', '26.4', '3.6', '20', '12.6', '16.4', '9.9',
       '59.6', '37.5', '2.7', '8.5', '4.8', '40.3', '19', '5.6', '18.9',
       '9.5', '7.9', '4.5', '9', '2.9', '20.6', '3.7', '4.2', '10.5',
       '10.2', '5.5', '15.2', '16.5', '28', '8.1', '49.3', '12.9', '14.5',
       '8.3', '4.9', '29', '8', '12.2', '29.4', '47.8', '2.8', '7.1',
       '11', '13.1', '37.2', '44.4', '17.5', '2.5', '10.3', '6.3', '10.4',
       '6.5', '13.4', '3.4', '27.9', '10.6', '27.3', '8.7', '17', '3.2',
       '6.8', '20.3', '10', '10.7', '9.7', '13.3', '36.8', '5.7', '7',
       '11.3', '25.6', '8.9', '20.1', '3.5', '7.8', '24.9', '16.1', '5.4',
       '41', '2.6', '30.7', '7.2', '21.2', '9.3', '38.9', '46.7', '51.3'],
      dtype=object)

In [6]:
#Just replace it with 2.5
merged_df["Undernourished"] = protein_dataset["Undernourished"].replace('<2.5','2.5')

In [7]:
#We can see that unit has been fixed, but undernourished is not a float
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 0 to 169
Data columns (total 100 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Country                        170 non-null    object 
 1   Alcoholic Beverages_kcal       170 non-null    float64
 2   Animal Products_kcal           170 non-null    float64
 3   Animal fats_kcal               170 non-null    float64
 4   Aquatic Products, Other_kcal   170 non-null    float64
 5   Cereals - Excluding Beer_kcal  170 non-null    float64
 6   Eggs_kcal                      170 non-null    float64
 7   Fish, Seafood_kcal             170 non-null    float64
 8   Fruits - Excluding Wine_kcal   170 non-null    float64
 9   Meat_kcal                      170 non-null    float64
 10  Milk - Excluding Butter_kcal   170 non-null    float64
 11  Miscellaneous_kcal             170 non-null    float64
 12  Offals_kcal                    170 non-null    fl

In [8]:
merged_df["Undernourished"] = pd.to_numeric((merged_df["Undernourished"]), downcast="float")
#It is now fixed
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 0 to 169
Data columns (total 100 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Country                        170 non-null    object 
 1   Alcoholic Beverages_kcal       170 non-null    float64
 2   Animal Products_kcal           170 non-null    float64
 3   Animal fats_kcal               170 non-null    float64
 4   Aquatic Products, Other_kcal   170 non-null    float64
 5   Cereals - Excluding Beer_kcal  170 non-null    float64
 6   Eggs_kcal                      170 non-null    float64
 7   Fish, Seafood_kcal             170 non-null    float64
 8   Fruits - Excluding Wine_kcal   170 non-null    float64
 9   Meat_kcal                      170 non-null    float64
 10  Milk - Excluding Butter_kcal   170 non-null    float64
 11  Miscellaneous_kcal             170 non-null    float64
 12  Offals_kcal                    170 non-null    fl

In [9]:
#Investigate missing values
missing_vals = merged_df.isnull().sum().sort_values(ascending = False)
missing_vals.head(7)

Active            8
Undernourished    7
Deaths            6
Recovered         6
Confirmed         6
Obesity           3
Sugar Crops_f     0
dtype: int64

In [10]:
'''
As we can see, several countries have missing values for certain columns. 
'''
missing_df = merged_df[merged_df.isna().any(axis=1)]
missing_df

,Country,Alcoholic Beverages_kcal,Animal Products_kcal,Animal fats_kcal,"Aquatic Products, Other_kcal",Cereals - Excluding Beer_kcal,Eggs_kcal,"Fish, Seafood_kcal",Fruits - Excluding Wine_kcal,Meat_kcal,...,Pulses_kg,Spices_kg,Starchy Roots_kg,Stimulants_kg,Sugar & Sweeteners_kg,Sugar Crops_kg,Treenuts_kg,Vegetable Oils_kg,Vegetables_kg,Vegetal Products_kg
4,Antigua and Barbuda,2.3041,15.3672,1.5429,0.0,13.7215,0.2057,1.7280,3.6824,7.0356,...,0.1840,0.1524,1.4522,0.1564,3.8749,0.0000,0.0253,0.8102,5.4495,33.3233
10,Bahamas,2.9145,14.7930,1.4205,0.0,10.7274,0.6613,1.2491,4.4575,8.1803,...,0.0504,0.0884,1.0547,0.1556,6.2151,0.0000,0.0495,0.5110,7.7649,38.5229
26,Canada,1.5748,12.6986,3.9656,0.0,13.0279,0.7731,0.5297,1.8182,4.7960,...,0.6304,0.0574,3.4940,0.4053,5.0237,0.0000,0.1439,1.3441,5.4961,33.9551
29,Chile,1.5942,12.3547,1.3783,0.0,19.5782,0.6310,0.3985,1.2786,7.4228,...,0.2677,0.0133,4.7183,0.1353,3.5582,0.0000,0.1190,0.5435,5.3092,34.9584
52,French Polynesia,1.4879,15.0676,1.8129,0.0,15.8885,0.4960,1.6419,1.0433,8.1409,...,0.1452,0.0111,3.5436,0.1260,2.8715,0.6809,0.0405,0.9675,4.0992,35.6559
59,Grenada,1.9979,12.2581,0.7284,0.0,13.7565,0.7284,1.1446,3.5588,5.7648,...,0.6417,0.3171,2.2222,0.1104,5.4480,0.0000,0.0143,0.9959,3.6085,37.8648
80,Kiribati,0.1145,5.9555,0.2291,0.0,16.9993,0.1309,2.6014,1.8815,2.6832,...,0.0032,0.0064,7.1830,0.0311,4.1373,0.0000,0.1951,0.4419,4.1389,41.1042
81,"Korea, North",1.4282,3.1273,0.0000,0.0,29.9926,0.4432,0.4186,1.6991,2.0438,...,1.1000,0.0000,6.2438,0.0142,0.3694,0.0000,0.0426,0.7783,12.8082,46.6046
105,Myanmar,0.1296,10.8663,0.8145,0.0,23.4358,0.3887,2.0548,1.3514,6.3680,...,1.2806,0.2170,1.4514,0.2298,2.8592,0.0000,0.2913,0.6322,7.0725,36.4807
109,New Caledonia,1.9087,13.3430,1.4361,0.0,15.9607,0.6362,0.8544,1.6906,6.5261,...,0.1275,0.0488,2.5648,0.1745,3.1901,0.0000,0.0376,1.3157,5.5941,38.2208


In [11]:
missing_df = protein_dataset[protein_dataset.isna().any(axis=1)]
missing_df

,Country,Alcoholic Beverages_p,Animal Products_p,Animal fats_p,"Aquatic Products, Other_p",Cereals - Excluding Beer_p,Eggs_p,"Fish, Seafood_p",Fruits - Excluding Wine_p,Meat_p,...,Vegetable Oils_p,Vegetables_p,Miscellaneous_p,Obesity,Undernourished,Confirmed,Deaths,Recovered,Active,Population
4,Antigua and Barbuda,0.1535,33.1901,0.1289,0.0000,10.5108,0.4850,8.2146,1.2586,16.0670,...,0.0430,1.6024,0.2947,19.1,NaN,0.293878,0.007143,0.190816,0.095918,98000.0
10,Bahamas,0.0325,34.1684,0.0895,0.0000,7.8081,1.6755,5.7340,1.8788,20.4311,...,0.0325,2.6759,1.1305,32.1,NaN,2.100763,0.044784,1.735115,0.320865,393000.0
26,Canada,0.3458,25.1828,0.6868,0.0049,12.9249,2.0553,2.8755,0.6868,13.7253,...,0.0840,1.5909,0.2915,31.3,<2.5,2.109961,0.054203,1.909848,NaN,38190000.0
29,Chile,0.2314,25.6899,0.1016,0.0000,17.7775,1.6141,2.0374,0.4966,16.5077,...,0.0169,1.4222,0.0734,28.8,2.7,3.842229,0.097047,3.626194,NaN,19470000.0
52,French Polynesia,0.3778,32.9981,0.2042,0.0153,10.7050,1.1741,7.0448,0.3931,18.8677,...,0.0153,1.2558,1.1129,NaN,4.2,NaN,NaN,NaN,NaN,280000.0
59,Grenada,0.1204,30.3273,0.0567,0.0000,12.2414,1.8631,5.9719,1.4452,15.3939,...,0.0283,0.7793,0.3684,20.2,NaN,0.130973,0.000885,0.129204,0.000885,113000.0
80,Kiribati,0.0338,24.9594,0.0473,0.0000,15.3680,0.4058,16.1052,0.9131,7.6502,...,0.0000,1.3731,0.1420,45.6,2.7,NaN,NaN,NaN,NaN,125000.0
81,"Korea, North",0.0287,10.0832,0.0000,0.0000,23.9070,1.3872,2.9561,0.6218,5.0990,...,0.0000,4.6398,0.0287,7.1,47.8,NaN,NaN,NaN,NaN,25779000.0
105,Myanmar,0.0161,23.9286,0.0911,0.0000,16.0006,0.8517,8.7797,0.4018,10.2743,...,0.0000,1.8213,0.0804,5.7,10.6,NaN,NaN,NaN,NaN,54704000.0
109,New Caledonia,0.3262,29.1790,0.1873,0.0181,13.6531,1.6915,3.9207,0.6585,15.3749,...,0.0362,1.7640,1.1841,NaN,7.1,NaN,NaN,NaN,NaN,295000.0


In [12]:
#Using john hopkins covid data, fill in the active cases as a percentage of population
merged_df['Active'] = np.where(merged_df['Country'] == 'Chile', 123835/merged_df.Population, merged_df.Active)
merged_df['Active'] = np.where(merged_df['Country'] == 'Canada', 89998/merged_df.Population, merged_df.Active)

In [13]:
'''
Unfortunately, we cannot estimate the covid data, because that's going to be related to our predictive outcomes.
Any estimation would just not be realistic.
For this reason, we drop the following countries:
'''
index_names = merged_df[ (merged_df['Country'] == 'Turkmenistan') | (merged_df['Country'] == 'French Polynesia') | (merged_df['Country'] == 'Korea, North') | (merged_df['Country'] == 'Kiribati') | (merged_df['Country'] == 'Myanmar') | (merged_df['Country'] == 'New Caledonia')].index
merged_df.drop(index_names, inplace = True) 

In [14]:
merged_na_df = merged_df[merged_df.isna().any(axis=1)]
merged_na_df['Undernourished']

4      NaN
10     NaN
59     NaN
124    NaN
128    NaN
129    NaN
147    3.5
148    NaN
Name: Undernourished, dtype: float32

In [15]:
#estimate the missing values using the knn inputer
from sklearn.impute import KNNImputer
#In order to use the knn inputer we need to have no text fields, so i'm storing the country names somewhere else
copy_df = merged_df.copy(deep=True)
copy_df.drop(copy_df.columns.difference(['Country','Population']), 1, inplace=True)
#drop the country frame to use the inputer
merged_df = merged_df.drop(['Country'],axis=1)
imputer = KNNImputer(n_neighbors=5) 
merged_df = pd.DataFrame(imputer.fit_transform(merged_df), columns=merged_df.columns)

In [16]:
#put the country names back where they belong
merged_df = merged_df.merge(copy_df, left_on='Population', right_on='Population')

In [17]:
merged_df.to_csv(r'data/merged_df.csv', index = False)

In [18]:
kcal_cols = [col for col in merged_df.columns if col.endswith('kcal')] + ['Country','Obesity', 'Undernourished', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Population']
data_kcal = merged_df[kcal_cols]
data_kcal.to_csv(r'data/data_clean/kcal_data_clean.csv', index = False)

fat_cols = [col for col in merged_df.columns if col.endswith('f')] + ['Country','Obesity', 'Undernourished', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Population']
data_fat = merged_df[fat_cols]
data_fat.to_csv(r'data/data_clean/fat_data_clean.csv', index = False)

protein_cols = [col for col in merged_df.columns if col.endswith('p')] + ['Country','Obesity', 'Undernourished', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Population']
data_protein = merged_df[protein_cols]
data_protein.to_csv(r'data/data_clean/protein_data_clean.csv', index = False)

kg_cols = [col for col in merged_df.columns if col.endswith('kg')] + ['Country','Obesity', 'Undernourished', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Population']
data_kg = merged_df[kg_cols]
data_kg.to_csv(r'data/data_clean/kg_data_clean.csv', index = False)